In [ ]:
!pip install numpy --upgrade
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm
import copy
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

In [ ]:
#Importing data
train=pd.read_csv('/content/drive/My Drive/Train (1).csv')
test=pd.read_csv('/content/drive/My Drive/Test (1).csv')

In [ ]:
true_values = []
for v in test.values:
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in test.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

In [ ]:
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

#A merger
train['combiner']='x'

#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)

#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]

In [ ]:
train['date1'] = train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
train['date2'] = train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
train['date3'] = train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
train.drop('join_date', axis=1, inplace=True)

test['date1'] = test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
test['date2'] = test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
test['date3'] = test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
test.drop('join_date', axis=1, inplace=True)

train['date_diff'] = train['date3'] - train['birth_year']
test['date_diff'] = test['date3'] - test['birth_year']

In [ ]:
train

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label,date1,date2,date3,date_diff
0,4WKQSBB,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0,1.0,2.0,2019.0,32.0
1,CP5S02H,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0,1.0,6.0,2019.0,38.0
2,2YKDILJ,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0,1.0,6.0,2013.0,22.0
3,2S9E81J,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0,1.0,8.0,2019.0,29.0
4,BHDYVFT,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0,1.0,8.0,2019.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,F,M,1974,748L,2A7I,T4MS,ECY3,8SBA93U X ECY3,0,1.0,9.0,2019.0,45.0
611768,Y73ETTO,M,M,1987,XX25,Q6J6,90QI,ECY3,Y73ETTO X ECY3,0,1.0,11.0,2018.0,31.0
611769,MVXG2LC,M,M,1985,BOAS,6PE7,T4MS,ECY3,MVXG2LC X ECY3,0,1.0,2.0,2019.0,34.0
611770,BUQ8DQN,M,M,1989,UAOD,2A7I,T4MS,ECY3,BUQ8DQN X ECY3,0,1.0,7.0,2019.0,30.0


In [ ]:
test

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,date1,date2,date3,date_diff
0,F86J5PC,M,M,1984,94KC,DZRV,90QI,P5DA,F86J5PC X P5DA,1.0,12.0,2018.0,34.0
1,H6141K3,M,M,1996,1X1H,J9SY,90QI,P5DA,H6141K3 X P5DA,1.0,10.0,2019.0,23.0
2,RBAYUXZ,F,W,1968,UAOD,2A7I,T4MS,P5DA,RBAYUXZ X P5DA,1.0,1.0,2020.0,52.0
3,KCBILBQ,M,M,1989,94KC,2A7I,T4MS,P5DA,KCBILBQ X P5DA,1.0,2.0,2019.0,30.0
4,LSEC1ZJ,F,M,1982,UAOD,0KID,T4MS,P5DA,LSEC1ZJ X P5DA,1.0,2.0,2020.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,0GMU5UH,F,M,1986,748L,2A7I,T4MS,ECY3,0GMU5UH X ECY3,1.0,4.0,2019.0,33.0
209996,I9W11CD,M,M,1980,UAOD,0KID,T4MS,ECY3,I9W11CD X ECY3,1.0,8.0,2019.0,39.0
209997,42WTEGT,M,M,1993,748L,SST3,56SI,ECY3,42WTEGT X ECY3,1.0,4.0,2018.0,25.0
209998,8EKC4O9,F,U,1981,49BM,2A7I,T4MS,ECY3,8EKC4O9 X ECY3,1.0,5.0,2015.0,34.0


In [ ]:
catFeatureslist=train[['occupation_category_code','occupation_code','branch_code','marital_status','sex','PCODE']]
catFeatureslist1=test[['occupation_category_code','occupation_code','branch_code','marital_status','sex','PCODE']]

In [ ]:
for cf1 in catFeatureslist:
    le = LabelEncoder()
    le.fit(train[cf1].unique())
    train[cf1] = le.transform(train[cf1])

In [ ]:
train

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label,date1,date2,date3,date_diff
0,4WKQSBB,0,1,1987,0,17,5,15,4WKQSBB X P5DA,0,1.0,2.0,2019.0,32.0
1,CP5S02H,0,1,1981,11,17,5,15,CP5S02H X P5DA,0,1.0,6.0,2019.0,38.0
2,2YKDILJ,1,5,1991,3,176,1,15,2YKDILJ X P5DA,0,1.0,6.0,2013.0,22.0
3,2S9E81J,1,1,1990,0,76,0,15,2S9E81J X P5DA,0,1.0,8.0,2019.0,29.0
4,BHDYVFT,1,1,1990,3,148,5,15,BHDYVFT X P5DA,0,1.0,8.0,2019.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,0,1,1974,3,17,5,5,8SBA93U X ECY3,0,1.0,9.0,2019.0,45.0
611768,Y73ETTO,1,1,1987,13,170,1,5,Y73ETTO X ECY3,0,1.0,11.0,2018.0,31.0
611769,MVXG2LC,1,1,1985,6,44,5,5,MVXG2LC X ECY3,0,1.0,2.0,2019.0,34.0
611770,BUQ8DQN,1,1,1989,11,17,5,5,BUQ8DQN X ECY3,0,1.0,7.0,2019.0,30.0


In [ ]:
for cf1 in catFeatureslist1:
    le = LabelEncoder()
    le.fit(test[cf1].unique())
    test[cf1] = le.transform(test[cf1])

In [ ]:
test

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,date1,date2,date3,date_diff
0,F86J5PC,1,2,1984,4,79,1,15,F86J5PC X P5DA,1.0,12.0,2018.0,34.0
1,H6141K3,1,2,1996,0,103,1,15,H6141K3 X P5DA,1.0,10.0,2019.0,23.0
2,RBAYUXZ,0,7,1968,11,18,5,15,RBAYUXZ X P5DA,1.0,1.0,2020.0,52.0
3,KCBILBQ,1,2,1989,4,18,5,15,KCBILBQ X P5DA,1.0,2.0,2019.0,30.0
4,LSEC1ZJ,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA,1.0,2.0,2020.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,0GMU5UH,0,2,1986,3,18,5,5,0GMU5UH X ECY3,1.0,4.0,2019.0,33.0
209996,I9W11CD,1,2,1980,11,3,5,5,I9W11CD X ECY3,1.0,8.0,2019.0,39.0
209997,42WTEGT,1,2,1993,3,156,0,5,42WTEGT X ECY3,1.0,4.0,2018.0,25.0
209998,8EKC4O9,0,6,1981,2,18,5,5,8EKC4O9 X ECY3,1.0,5.0,2015.0,34.0


In [ ]:
# retrieve just the numeric input values
num_cols = ['sex',"marital_status","birth_year", 'branch_code','occupation_code','PCODE', 'occupation_category_code','date1','date2','date3','date_diff']

In [ ]:
# perform a robust scaler transform of the dataset
trans = RobustScaler()
train[num_cols] = trans.fit_transform(train[num_cols])

In [ ]:
# perform a robust scaler transform of the dataset
trans = RobustScaler()
test[num_cols] = trans.fit_transform(test[num_cols])

In [ ]:
train

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label,date1,date2,date3,date_diff
0,4WKQSBB,-1.0,0.0,0.500000,-0.571429,-0.081761,0.00,0.5,4WKQSBB X P5DA,0,0.0,-0.8,0.5,-0.428571
1,CP5S02H,-1.0,0.0,0.071429,1.000000,-0.081761,0.00,0.5,CP5S02H X P5DA,0,0.0,0.0,0.5,0.000000
2,2YKDILJ,0.0,4.0,0.785714,-0.142857,0.918239,-1.00,0.5,2YKDILJ X P5DA,0,0.0,0.0,-2.5,-1.142857
3,2S9E81J,0.0,0.0,0.714286,-0.571429,0.289308,-1.25,0.5,2S9E81J X P5DA,0,0.0,0.4,0.5,-0.642857
4,BHDYVFT,0.0,0.0,0.714286,-0.142857,0.742138,0.00,0.5,BHDYVFT X P5DA,0,0.0,0.4,0.5,-0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,8SBA93U,-1.0,0.0,-0.428571,-0.142857,-0.081761,0.00,-0.5,8SBA93U X ECY3,0,0.0,0.6,0.5,0.500000
611768,Y73ETTO,0.0,0.0,0.500000,1.285714,0.880503,-1.00,-0.5,Y73ETTO X ECY3,0,0.0,1.0,0.0,-0.500000
611769,MVXG2LC,0.0,0.0,0.357143,0.285714,0.088050,0.00,-0.5,MVXG2LC X ECY3,0,0.0,-0.8,0.5,-0.285714
611770,BUQ8DQN,0.0,0.0,0.642857,1.000000,-0.081761,0.00,-0.5,BUQ8DQN X ECY3,0,0.0,0.2,0.5,-0.571429


In [ ]:
test

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,date1,date2,date3,date_diff
0,F86J5PC,0.0,0.0,0.285714,0.000,0.408,-1.00,0.5,F86J5PC X P5DA,0.0,1.2,0.0,-0.214286
1,H6141K3,0.0,0.0,1.142857,-0.500,0.600,-1.00,0.5,H6141K3 X P5DA,0.0,0.8,0.5,-1.000000
2,RBAYUXZ,-1.0,5.0,-0.857143,0.875,-0.080,0.00,0.5,RBAYUXZ X P5DA,0.0,-1.0,1.0,1.071429
3,KCBILBQ,0.0,0.0,0.642857,0.000,-0.080,0.00,0.5,KCBILBQ X P5DA,0.0,-0.8,0.5,-0.500000
4,LSEC1ZJ,-1.0,0.0,0.142857,0.875,-0.200,0.00,0.5,LSEC1ZJ X P5DA,0.0,-0.8,1.0,0.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,0GMU5UH,-1.0,0.0,0.428571,-0.125,-0.080,0.00,-0.5,0GMU5UH X ECY3,0.0,-0.4,0.5,-0.285714
209996,I9W11CD,0.0,0.0,0.000000,0.875,-0.200,0.00,-0.5,I9W11CD X ECY3,0.0,0.4,0.5,0.142857
209997,42WTEGT,0.0,0.0,0.928571,-0.125,1.024,-1.25,-0.5,42WTEGT X ECY3,0.0,-0.4,0.0,-0.857143
209998,8EKC4O9,-1.0,4.0,0.071429,-0.250,-0.080,0.00,-0.5,8EKC4O9 X ECY3,0.0,-0.2,-1.5,-0.214286


In [ ]:
X_train=train.drop(columns=['ID', 'Label', 'ID X PCODE'],axis=1)

In [ ]:
y_train=train['Label']

In [ ]:
X_test=test.drop(columns=['ID', 'ID X PCODE'], axis=1)

In [ ]:
from sklearn import model_selection

In [ ]:
y_train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
y_train.mean()

0.1084603414343906

In [ ]:
y_train.fillna(0.1084603414343906,inplace=True)

In [ ]:

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
X_train.fillna(0,inplace=True)

In [ ]:
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
X_test.fillna(0,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier

In [ ]:

# Initializing models

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = XGBClassifier()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

params = {'kneighborsclassifier__n_neighbors': [1, 5],
          'randomforestclassifier__n_estimators': [10, 50],
          'meta_classifier__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sclf, 
                    param_grid=params, 
                    cv=5,
                    refit=True)
grid.fit(X_train, y_train)

cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)

In [ ]:
y_pred=sclf.predict_proba(X_test)[:,1]

In [ ]:
'''accuracy = accuracy_score(y_test1, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))'''

In [ ]:
y_pred

In [ ]:
df_answer = test[['ID X PCODE']]
df_answer['Label'] = y_pred

In [ ]:
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0

df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission3.csv', index=False)